In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.linalg import lstsq
import scipy.linalg as linalg 
from scipy.linalg import solve
from scipy.stats import norm
import math

In [ ]:
def America_put(S0,T,M,u,d,X,r,p):
    delta = T/M
    price = pd.DataFrame(columns = np.arange(M+1),index = np.arange(M+1))
    price.loc[0,0] = S0

    for i in price.columns[1:]:
        for k in range(i+1):
            price.loc[k,i] = S0 * u**(i-k) * d**k
            
    # Put payoff
    payoff = X-price
    payoff = pd.DataFrame(np.where(payoff>0,payoff,0))
    for i in np.linspace(M-1,0,M).astype(int):
        for j in range(i+1):
            # caclulated backward
            CV = np.exp(-r*delta) * (p*payoff.loc[j,i+1] + (1-p) * payoff.loc[j+1,i+1])
            payoff.loc[j,i] = max(CV,payoff.loc[j,i])
    return(payoff.loc[0,0])

In [ ]:
def A_para(r,sigma,T,M):
    delta = T/M
    c = 0.5 * (np.exp(-r * delta) + np.exp((r+sigma**2) * delta))
    d = c - np.sqrt(c**2-1)
    u = 1/d
    p = (np.exp(r*delta) - d)/(u-d)
    return([u,d,p])
def B_para(r,sigma,T,M):
    delta = T/M
    d = np.exp((r-sigma**2/2)*delta - sigma * np.sqrt(delta))
    u = np.exp((r-sigma**2/2)*delta + sigma * np.sqrt(delta))
    p = 0.5
    return([u,d,p])

In [ ]:
result_1 = pd.DataFrame(index = [20,40,80,100,200,500], columns = ['MethodA','MethodB'])
for n in result_1.index:
    parameters_A = A_para(r=0.055, sigma = 0.25, T = 0.5, M = n)
    result_1.loc[n,'MethodA'] = America_put(S0=180,X=180,u = parameters_A[0], d =parameters_A[1],p=parameters_A[2],T = 0.5, M=n,r = 0.055)
    parameters_B = B_para(r=0.055, sigma = 0.25, T = 0.5, M = n)
    result_1.loc[n,'MethodB'] = America_put(S0=180,X=180,u = parameters_B[0], d =parameters_B[1],p=parameters_B[2],T = 0.5, M=n,r = 0.055)
result_1

In [ ]:
result_1

In [ ]:
def CRR_para(r,sigma,T,n):
    if T <= 0 :
        u = 0
        d = 0
        p = 0
    else:
        delta = T/n
        d = np.exp(-sigma * np.sqrt(delta))
        u = np.exp(sigma * np.sqrt(delta))
        p = (np.exp(r*delta) - d)/(u-d)
    return([u,d,p])

In [ ]:
# Option Greeks
S0_list = np.arange(170,190+2,2)
Delta_S = []
epsilon = 0.00001
CRR_para1 = CRR_para(r=0.055, sigma = 0.25, T = 0.5, n = 50)
for i in range(len(S0_list)):
    P_old = America_put(S0=S0_list[i]-epsilon,X=180,u = CRR_para1[0], d =CRR_para1[1],p=CRR_para1[2],T = 0.5, n=50,r = 0.055)
    P_new = America_put(S0=S0_list[i]+epsilon,X=180,u = CRR_para1[0], d =CRR_para1[1],p=CRR_para1[2],T = 0.5, n=50,r = 0.055)
    Delta_S.append((P_new-P_old)/(2*epsilon))
T_list = np.arange(0, 0.18 + 0.003, 0.003)
Delta_T = []
for t in T_list:
    CRR_para1 = CRR_para(r=0.055, sigma = 0.25, T = t, n = 50)
    P_old = America_put(S0=180-epsilon,X=180,u = CRR_para1[0], d =CRR_para1[1],p=CRR_para1[2],T =t, n=50,r = 0.055)
    P_new = America_put(S0=180+epsilon,X=180,u = CRR_para1[0], d =CRR_para1[1],p=CRR_para1[2],T =t, n=50,r = 0.055)
    Delta_T.append((P_new-P_old)/(2*epsilon))
Theta_T = []
for i in range(len(T_list))[1:]:
    CRR_para1 = CRR_para(r=0.055, sigma = 0.25, T = T_list[i]-epsilon, n = 50)
    CRR_para2 = CRR_para(r=0.055, sigma = 0.25, T = T_list[i]+epsilon, n = 50)
    P_old = America_put(S0=180,X=180,u = CRR_para1[0], d =CRR_para1[1],p=CRR_para1[2],T = T_list[i]-epsilon, n=50,r = 0.055)
    P_new = America_put(S0=180,X=180,u = CRR_para2[0], d =CRR_para2[1],p=CRR_para2[2],T = T_list[i]+epsilon, n=50,r = 0.055)
    Theta_T.append(-(P_new-P_old)/(2*epsilon))
Vega_S = []
for i in range(len(S0_list)):
    CRR_para1 = CRR_para(r=0.055, sigma = 0.25-epsilon, T = 0.5, n = 50)
    CRR_para2 = CRR_para(r=0.055, sigma = 0.25+epsilon, T = 0.5, n = 50)
    P_old = America_put(S0=S0_list[i],X=180,u = CRR_para1[0], d =CRR_para1[1],p=CRR_para1[2],T = 0.5, n=50,r = 0.055)
    P_new = America_put(S0=S0_list[i],X=180,u = CRR_para2[0], d =CRR_para2[1],p=CRR_para2[2],T = 0.5, n=50,r = 0.055)
    Vega_S.append((P_new-P_old)/(2*epsilon))

# Trinomial Tree Option price

In [ ]:
def Tri_A_para(r,sigma,T,n):
    delta = T/n
    d = np.exp(-sigma*np.sqrt(3*delta))
    u = 1/d
    p_d = (r*delta *(1-u) + (r*delta)**2 + sigma**2*delta)/((u-d)*(1-d))
    pu = (r*delta *(1-d) + (r*delta)**2 + sigma**2*delta)/((u-d)*(u-1))
    return([u,d,pu,p_d])
def Tri_B_para(r,sigma,T,n):
    delta = T/n
    dXu = sigma * np.sqrt(3*delta)
    dXd = -sigma * np.sqrt(3*delta)
    LU = (sigma**2 * delta) + ((r-sigma**2/2)**2 * delta **2)
    RU = (r-sigma**2/2) * delta
    p_d = 0.5 * (LU/dXu**2 - RU/dXu)
    pu = 0.5 * (LU/dXu**2 + RU/dXu)
    return([dXu,dXd,pu,p_d])

In [ ]:
def Tri_put(S0,T,n,u,d,X,r,p_u,p_d):
    delta = T/n
    pm = 1-p_d-p_u
    m=1
    price = pd.DataFrame(columns = np.arange(n+1),index = np.arange(n+1))
    price.loc[0,0] = S0
    for i in price.columns[1:]:
        for k in range(2*i+1):
            u_p = max(i-k,0)
            d_p = max(k-i,0)
            price.loc[k,i] = S0 * (u ** u_p) * (d ** d_p)
    payoff = X-price
    payoff = pd.DataFrame(np.where(payoff>0,payoff,0))
    for i in np.linspace(n-1,0,n).astype(int):
        for j in range(2*i+1):
            CV = np.exp(-r*delta) * (p_u*payoff.loc[j,i+1]+ pm*payoff.loc[j+1,i+1] + p_d * payoff.loc[j+2,i+1])
            payoff.loc[j,i] = max(CV,payoff.loc[j,i])
    return(payoff.loc[0,0])

In [ ]:
def Tri_put_log(S0,T,n,dXu,dXd,X,r,p_u,p_d):
    delta = T/n
    pm = 1-p_d-p_u
    price = pd.DataFrame(columns = np.arange(n+1),index = np.arange(n+1))
    X0 = np.log(S0)
    price.loc[0,0] = X0
    for i in price.columns[1:]:
        for k in range(2*i+1):
            u_p = max(i-k,0)
            d_p = max(k-i,0)
            price.loc[k,i] = np.exp(X0 + u_p * dXu + d_p * dXd)
    price.loc[0,0] = S0
    payoff = X-price
    payoff = pd.DataFrame(np.where(payoff>0,payoff,0))
    for i in np.linspace(n-1,0,n).astype(int):
        for j in range(2*i+1):
            CV = np.exp(-r*delta) * (p_u*payoff.loc[j,i+1]+ pm*payoff.loc[j+1,i+1] + p_d * payoff.loc[j+2,i+1])
            payoff.loc[j,i] = max(CV,payoff.loc[j,i])
    return(payoff.loc[0,0])